In [ ]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq

from langchain_community.document_loaders import WebBaseLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

e:\RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
load_dotenv()
groq_api_key = os.environ["GROQ_API_KEY"]

In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size  = 1000,
    chunk_overlap = 200
)
final_documents = text_splitter.split_documents(docs[:50])

In [7]:
embeddings = OllamaEmbeddings(model ="nomic-embed-text")
vectorstore = FAISS.from_documents(final_documents,embeddings)
retriver = vectorstore.as_retriever()

In [17]:
llm = ChatGroq(
    groq_api_key = groq_api_key,
    model_name ="llama-3.3-70b-versatile"
)

In [18]:
prompt = ChatPromptTemplate.from_template(
    """

    answer the question based only on the provided context.
    be precise and factual.

    <context>
    {context}
    </context>

    Question: {input}
    """
)

In [21]:
document_chain = create_stuff_documents_chain(llm,prompt)
retrieval_chain = create_retrieval_chain(retriver,document_chain)

query = "is langchain free to use ?"

response = retrieval_chain.invoke({"input":query})

print("\n question:")
print(query)

print("answer:")
print(response["answer"])



 question:
is langchain free to use ?
answer:
The provided context does not explicitly state that LangChain is free to use. However, it does mention that creating an account on smith.langchain.com does not require a credit card.
